# Analyze data with Apache Spark

https://learn.microsoft.com/en-ca/azure/synapse-analytics/spark/apache-spark-data-visualization-tutorial

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("spark://spark:7077") \
        .appName("jupyter-notebook-index2mongo") \
        .config("spark.driver.memory", "1g") \
        .getOrCreate()

In [4]:
spark

In [3]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")  

# Spark SQL

In [5]:
sdf = spark.read.option("overwriteSchema", "true").parquet('/home/jovyan/work/data/**/*.parquet.zst')

In [6]:
type(sdf)

pyspark.sql.dataframe.DataFrame

In [16]:
sdf.tail(10)

[Row(language='FR', country='FR', job_id='379974/15/185782', productCategoryAlt=None, productAltEAN=None, productPriceBase=None, productBrand='PTIT BASILE', productCategory="['Tout pour bébé', 'Puériculture', 'Chambre bébé']", productDate='2022-12-29T00:25:33Z', crawlID='417ca1d7-3878-437d-a6a9-13d82f4ced78', currentURL='https://www.auchan.fr/bebe/puericulture/chambre-bebe/ca-7135359?page=71', productLinkDetail='https://www.auchan.fr/ptit-basile-parure-housse-de-couette-bio-brodee-taie-d-oreiller-nuages/pr-C901112', productEAN='3760240961222', productID='108146', productAltId=None, productImage='https://media.auchan.fr/148fd2a8-da43-4456-ac8b-047fe407c853_460x460/B2CD/', productIsAvailable=1, productIsBio=0, productIsFresh=0, productIsFreezer=0, productIsNew=0, promotion=0, isSponsored=None, productLotPriceBase=None, productLotPrice=None, productName="Parure housse de couette Bio brodée + Taie d'oreiller Nuages", productNutriscore=None, other='{"is_heard_on_radio": false}', productPack

# Pandas spark flavor

In [8]:
import pyspark.pandas as ps

In [13]:
psdf = ps.read_parquet('/home/jovyan/work/data/**/*.parquet.zst', index_col="productID")

In [14]:
type(psdf)

pyspark.pandas.frame.DataFrame

In [15]:
psdf.tail(10)

,language,country,job_id,productCategoryAlt,productAltEAN,productPriceBase,productBrand,productCategory,productDate,crawlID,currentURL,productLinkDetail,productEAN,productAltId,productImage,productIsAvailable,productIsBio,productIsFresh,productIsFreezer,productIsNew,promotion,isSponsored,productLotPriceBase,productLotPrice,productName,productNutriscore,other,productPackaging,typeCrawler,productPosition,productPricePrevious,productPrice,productPromotionText,productQuantityRating,productRating,enseigneID,enseigneName,productService,shopID,productStock,productUnit,productValueUnit,variant_master,variant_position,zip_code
productID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
108146,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:33Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-basile-parure-houss...,3760240961222,None,https://media.auchan.fr/148fd2a8-da43-4456-ac8...,1,0,0,0,0,0,NaN,NaN,NaN,Parure housse de couette Bio brodée + Taie d'o...,NaN,"{""is_heard_on_radio"": false}",None,WEB,41,NaN,46.599998,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
CA1306558,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:34Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-basile-drap-plat-be...,None,None,https://media.auchan.fr/1dc5de90-6051-4f0c-afb...,1,0,0,0,0,0,NaN,NaN,NaN,Drap plat bébé coton Bio 118x180 cm imprimé,NaN,"{""is_heard_on_radio"": false}",2 coloris,WEB,42,NaN,26.000000,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
108703,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:42Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-basile-gigoteuse-et...,3760240962687,None,https://media.auchan.fr/469fa5b2-ba27-47c8-b66...,1,0,0,0,0,0,NaN,NaN,NaN,Gigoteuse été Bio 6-24 mois Little sweet dreams,NaN,"{""is_heard_on_radio"": false}",None,WEB,48,NaN,36.900002,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
269862,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:42Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-basile-gigoteuse-et...,3760240962199,None,https://media.auchan.fr/8768feae-8033-45a2-acc...,1,0,0,0,0,0,NaN,NaN,NaN,Gigoteuse été en coton bio 6-36 mois Pluie d'é...,NaN,"{""is_heard_on_radio"": false}",None,WEB,49,NaN,38.900002,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
727591,FR,FR,379974/15/185782,NaN,NaN,NaN,Bébé Provence,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:46Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/bebe-provence-matelas-be...,None,None,https://media.auchan.fr/0ae5f99d-8843-4ad5-875...,1,0,0,0,0,1,NaN,NaN,NaN,Matelas bébé mousse HD 60 x 120 cm BELLE NUIT,NaN,"{""is_heard_on_radio"": false}",None,WEB,51,NaN,109.000000,25% cagnotté,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
221894,FR,FR,379974/15/185782,NaN,NaN,NaN,Ptit Albatros,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:47Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-albatros-p-tit-alba...,3454980150250,None,https://media.auchan.fr/433deab3-860e-4e78-85f...,1,0,0,0,0,0,NaN,NaN,NaN,P'tit Albatros - Plan Incliné Bébé 60x35 cm - ...,NaN,"{""is_heard_on_radio"": false}",None,WEB,50,NaN,19.900000,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
CA1306562,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",202